In [14]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [12]:
import pandas as pd

# Constants
grid_size = 5
spacing_xy = 48.8
spacing_z = 27.112
half_shift = spacing_xy / 4

# Board ID mapping
board_ids = {
    0: 'E1',   1: 'O10', 2: 'E2',  3: 'O2',  4: 'E3',
    5: 'O3',   6: 'E9',  7: 'O9',  8: 'E5',  9: 'O5',
    10: 'E6', 11: 'O6', 12: 'E7', 13: 'O7', 14: 'E8'
}

# CAEN-to-layer mapping
caen_mapping = {
    0: [(0, 24, 0), (25, 49, 1), (50, 62, '4a')],
    1: [(0, 24, 2), (25, 49, 3), (50, 61, '4b')],
    2: [(0, 24, 5), (25, 49, 6), (50, 62, '9a')],
    3: [(0, 24, 7), (25, 49, 8), (50, 61, '9b')],
    4: [(0, 24, 10), (25, 49, 11), (50, 62, '14a')],
    5: [(0, 24, 12), (25, 49, 13), (50, 61, '14b')],
}

records = []

for caen, ranges in caen_mapping.items():
    for start_ch, end_ch, layer in ranges:
        num_channels = end_ch - start_ch + 1

        if isinstance(layer, str) and layer.endswith('a'):
            position_offset = 0
            layer_number = int(layer[:-1])
        elif isinstance(layer, str) and layer.endswith('b'):
            position_offset = 13
            layer_number = int(layer[:-1])
        else:
            position_offset = 0
            layer_number = layer

        for i in range(num_channels):
            caen_channel = start_ch + i
            position = position_offset + i

            # Grid position
            row = position // grid_size
            col = position % grid_size

            # Centered x,y
            x = (2 - col) * spacing_xy
            y = (row - 2) * spacing_xy

            # Apply shift for staggered layers
            if layer_number % 2 == 0:
                x += half_shift
                y += half_shift
            else:
                x -= half_shift
                y -= half_shift

            z = layer_number * spacing_z
            channel_num = caen_channel + 64 * caen
            board_id = board_ids[layer_number]

            records.append({
                'caen': caen,
                'caen_channel': caen_channel,
                'Ch#': channel_num,
                'layer': layer_number,
                'brd_channel': position,
                'x': x,
                'y': y,
                'z': z,
                'board_id': board_id
            })

# Create DataFrame
df = pd.DataFrame.from_records(records)

# Optional: sort by Ch# or any other field
df = df.sort_values(by='Ch#').reset_index(drop=True)

print(df)


     caen  caen_channel  Ch#  layer  brd_channel      x      y        z  \
0       0             0    0      0            0  109.8  -85.4    0.000   
1       0             1    1      0            1   61.0  -85.4    0.000   
2       0             2    2      0            2   12.2  -85.4    0.000   
3       0             3    3      0            3  -36.6  -85.4    0.000   
4       0             4    4      0            4  -85.4  -85.4    0.000   
5       0             5    5      0            5  109.8  -36.6    0.000   
6       0             6    6      0            6   61.0  -36.6    0.000   
7       0             7    7      0            7   12.2  -36.6    0.000   
8       0             8    8      0            8  -36.6  -36.6    0.000   
9       0             9    9      0            9  -85.4  -36.6    0.000   
10      0            10   10      0           10  109.8   12.2    0.000   
11      0            11   11      0           11   61.0   12.2    0.000   
12      0            12  

In [13]:
df.to_pickle("geometry.pkl")